In [1]:
import pandas as pd
import inflection


In [2]:
df = pd.read_csv("C:\\Users\\jatos\\Desktop\\JA\\Material Data Science\Datasets\\archive\\zomato.csv")
df1 = df.copy()

In [3]:
# Funções

#Preenchimento do nome dos países:
COUNTRIES = {
1: "India",
14: "Australia",
30: "Brazil",
37: "Canada",
94: "Indonesia",
148: "New Zeland",
162: "Philippines",
166: "Qatar",
184: "Singapure",
189: "South Africa",
191: "Sri Lanka",
208: "Turkey",
214: "United Arab Emirates",
215: "England",
216: "United States of America",
}
def country_name(country_id):
    return COUNTRIES[country_id]

#-------------------------------------------------------
#Criação do Tipo de Categoria de Comida

def create_price_tye(price_range):
    if price_range == 1:
        return "cheap"
    elif price_range == 2:
        return "normal"
    elif price_range == 3:
        return "expensive"
    else:
        return "gourmet"

#--------------------------------------------------------
# Criação do nome das Cores

COLORS = {
"3F7E00": "darkgreen",
"5BA829": "green",
"9ACD32": "lightgreen",
"CDD614": "orange",
"FFBA00": "red",
"CBCBC8": "darkred",
"FF7800": "darkred",
}
def color_name(color_code):
    return COLORS[color_code]

#--------------------------------------------
# Renomear as colunas do DataFrame:

def rename_columns(dataframe):
    df1 = dataframe.copy()
    title = lambda x: inflection.titleize(x)
    snakecase = lambda x: inflection.underscore(x)
    spaces = lambda x: x.replace(" ", "")
    cols_old = list(df.columns)
    cols_old = list(map(title, cols_old))
    cols_old = list(map(spaces, cols_old))
    cols_new = list(map(snakecase, cols_old))
    df.columns = cols_new
    return df1
# Limpando os dados
df1 = df1.dropna(subset=['Cuisines'])
df1["Cuisines"] = df1.loc[:, "Cuisines"].apply(lambda x: x.split(",")[0])

# Convertendo a coluna dos códigos
df1['Country Code'] = df1['Country Code'].apply(lambda x: country_name(x))

In [5]:
# 1. Qual o nome do restaurante que possui a maior quantidade de avaliações?
df_aux = (df1.loc[:, ['Restaurant Name', 'Aggregate rating']]
          .groupby('Restaurant Name').count()
          .sort_values('Aggregate rating', ascending = False).reset_index())
print(df_aux.iloc[0,0])

Domino's Pizza


In [6]:
# 2. Qual o nome do restaurante com a maior nota média?
df_aux = (df1.loc[:, ['Restaurant Name', 'Aggregate rating']]
          .groupby('Restaurant Name').mean()
          .sort_values('Aggregate rating', ascending = False).reset_index())
print(df_aux.iloc[0,0])

Phil's Bbq


In [7]:
# 3. Qual o nome do restaurante que possui o maior valor de uma prato para duas pessoas?
df_aux = (df1.loc[:, ['Restaurant Name', 'Average Cost for two']]
          .groupby('Restaurant Name').max()
          .sort_values('Average Cost for two', ascending = False).reset_index())
pd.set_option('float_format', '{:,.2f}'.format)
print(df_aux.iloc[0,0])

d'Arry's Verandah Restaurant


In [8]:
# 4. Qual o nome do restaurante de tipo de culinária brasileira que possui a menor média de avaliação?
linha = df1['Cuisines'] == 'Brazilian'
df_aux = df1.loc[linha, :]
df_aux1 = df_aux.loc[:, ['Restaurant Name', 'Aggregate rating']].groupby('Restaurant Name').min().sort_values('Aggregate rating').reset_index()
print(df_aux1.iloc[0,0])


Restaurante dos Amigos


In [9]:
# 5. Qual o nome do restaurante de tipo de culinária brasileira, e que é do Brasil, que possui a maior média de avaliação?
linha = (df1['Cuisines'] == 'Brazilian') & (df1['Country Code'] == 'Brazil')
df_aux = df1.loc[linha, :]
df_aux1 = df_aux.loc[:, ['Restaurant Name', 'Aggregate rating']].groupby('Restaurant Name').max().sort_values('Aggregate rating', ascending = False).reset_index()
print(df_aux1.iloc[0,0])

Braseiro da Gávea


In [8]:
# 6. Os restaurantes que aceitam pedido online são também, na média, os restaurantes que mais possuem avaliações registradas?
valor_1 = (df1.loc[:, ['Aggregate rating','Restaurant ID','Has Online delivery']]
           .groupby('Has Online delivery').agg({'Restaurant ID': ['median','count']}).reset_index())
valor_1

Has Online delivery Restaurant ID      
                             median count
0                   0    16598168.0  4859
1                   1     3400868.0  2653

In [9]:
# 7. Os restaurantes que fazem reservas são também, na média, 
#os restaurantes que possuem o maior valor médio de um prato para duas pessoas?
resp = (df1.loc[:, ['Has Table booking', 'Average Cost for two']]
        .groupby('Has Table booking').mean()
        .sort_values('Average Cost for two', ascending = False).reset_index())
display(resp)

,Has Table booking,Average Cost for two
0,1,62931.512821
1,0,3460.612436


In [17]:
# 8. Os restaurantes do tipo de culinária japonesa dos Estados Unidos da América 
# possuem um valor médio de prato para duas pessoas maior que as churrascarias americanas (BBQ)
linha = (df1['Cuisines'] == 'Japanese') | (df1['Cuisines'] == 'BBQ')
df_aux = df1.loc[linha,:]
linha1 = df_aux['Country Code'] == 'United States of America'
df_aux1 = df_aux.loc[linha1,:]
resp = (df_aux1.loc[:, ['Average Cost for two', 'Cuisines']]
        .groupby('Cuisines').mean()
        .sort_values('Average Cost for two', ascending = False).reset_index())
resp

,Cuisines,Average Cost for two
0,Japanese,56.406250
1,BBQ,39.642857
